In [2]:
import pandas as pd
from datetime import datetime
import os
import csv

In [2]:
price_usd = pd.read_csv('../input/btc_usd.csv')
trades = pd.read_csv('../temporary/trades_combined.csv')
trades

,date,tid,price,amount,Currency
0,1664328207,56510490,98325.52,0.001017,AED
1,1664278646,56507029,72718.80,0.233777,AED
2,1664262787,56506374,103802.28,0.000963,AED
3,1664213474,56503957,67281.57,0.460000,AED
4,1664036001,56496319,66769.49,0.030000,AED
...,...,...,...,...,...
43024557,1546936993,2609678,5141000.51,0.007294,VES
43024558,1546937422,2609679,5169989.25,0.001934,VES
43024559,1546937551,2609680,4993996.35,0.004606,VES
43024560,1546938200,2609681,5280099.85,0.044980,VES


In [3]:
price_usd = pd.read_csv('../input/btc_usd.csv')
price_usd['date'] = pd.to_datetime(price_usd['Date'])
price_usd['date'] = pd.to_datetime(price_usd['date'].dt.strftime('%Y-%m-%d'))
price_usd = price_usd[['date', 'Close']]
price_usd

,date,Close
0,2017-02-01,989.023010
1,2017-02-02,1011.799988
2,2017-02-03,1029.910034
3,2017-02-04,1042.900024
4,2017-02-05,1027.339966
...,...,...
2095,2022-10-28,20595.351563
2096,2022-10-29,20818.476563
2097,2022-10-30,20635.603516
2098,2022-10-31,20495.773438


In [4]:
trades['date'] = pd.to_datetime(trades['date'], unit = 's')
trades

,date,tid,price,amount,Currency
0,2022-09-28 01:23:27,56510490,98325.52,0.001017,AED
1,2022-09-27 11:37:26,56507029,72718.80,0.233777,AED
2,2022-09-27 07:13:07,56506374,103802.28,0.000963,AED
3,2022-09-26 17:31:14,56503957,67281.57,0.460000,AED
4,2022-09-24 16:13:21,56496319,66769.49,0.030000,AED
...,...,...,...,...,...
43024557,2019-01-08 08:43:13,2609678,5141000.51,0.007294,VES
43024558,2019-01-08 08:50:22,2609679,5169989.25,0.001934,VES
43024559,2019-01-08 08:52:31,2609680,4993996.35,0.004606,VES
43024560,2019-01-08 09:03:20,2609681,5280099.85,0.044980,VES


In [5]:
trades = trades.sort_values(by='date')
trades = trades.drop_duplicates(subset = ['tid']) #drop duplicates
trades = trades[trades['date'] > '2017-03-15'] #only use trades March 15, 2017 and beyond
trades

,date,tid,price,amount,Currency
10649809,2017-03-15 00:00:13,6277278,26490.07,0.075500,MXN
40515110,2017-03-15 00:00:15,100281,4126984.13,0.031500,VEF
39172389,2017-03-15 00:00:15,6277265,1292.97,0.657400,USD
7852718,2017-03-15 00:00:24,6277292,1068.00,0.110000,GBP
7852716,2017-03-15 00:00:40,6277296,1062.93,0.117600,GBP
...,...,...,...,...,...
3154199,2022-09-28 01:55:04,56510551,84041264.26,0.002856,COP
36360044,2022-09-28 01:55:10,56510558,18029.58,0.031892,USD
35810034,2022-09-28 01:55:10,56510556,21699.47,0.005530,USD
35810035,2022-09-28 01:57:02,56510553,20045.22,0.002494,USD


In [6]:
#Add nominal amount of crypto bought, both in domestic currency and USD
trades['amount_nominal'] = trades['price']*trades['amount']
trades = pd.merge_asof(trades, price_usd, on = 'date')
trades['amount_usd'] = trades['amount']*trades['Close']
trades = trades.rename(columns = {'Close': 'crypto_rate_usd', 'Currency':'currency'})
# trades = trades.drop(columns = ['Close'])
trades

,date,tid,price,amount,Currency,amount_nominal,Close,amount_usd
0,2017-03-15 00:00:13,6277278,26490.07,0.075500,MXN,2000.000285,1249.609985,94.345554
1,2017-03-15 00:00:15,100281,4126984.13,0.031500,VEF,130000.000095,1249.609985,39.362715
2,2017-03-15 00:00:15,6277265,1292.97,0.657400,USD,849.998478,1249.609985,821.493604
3,2017-03-15 00:00:24,6277292,1068.00,0.110000,GBP,117.480000,1249.609985,137.457098
4,2017-03-15 00:00:40,6277296,1062.93,0.117600,GBP,125.000568,1249.609985,146.954134
...,...,...,...,...,...,...,...,...
42636057,2022-09-28 01:55:04,56510551,84041264.26,0.002856,COP,239999.999998,19426.720703,55.477663
42636058,2022-09-28 01:55:10,56510558,18029.58,0.031892,USD,575.000087,19426.720703,619.557754
42636059,2022-09-28 01:55:10,56510556,21699.47,0.005530,USD,120.000022,19426.720703,107.431514
42636060,2022-09-28 01:57:02,56510553,20045.22,0.002494,USD,49.999995,19426.720703,48.457235


In [7]:
trades

,date,tid,price,amount,Currency,amount_nominal,Close,amount_usd
0,2017-03-15 00:00:13,6277278,26490.07,0.075500,MXN,2000.000285,1249.609985,94.345554
1,2017-03-15 00:00:15,100281,4126984.13,0.031500,VEF,130000.000095,1249.609985,39.362715
2,2017-03-15 00:00:15,6277265,1292.97,0.657400,USD,849.998478,1249.609985,821.493604
3,2017-03-15 00:00:24,6277292,1068.00,0.110000,GBP,117.480000,1249.609985,137.457098
4,2017-03-15 00:00:40,6277296,1062.93,0.117600,GBP,125.000568,1249.609985,146.954134
...,...,...,...,...,...,...,...,...
42636057,2022-09-28 01:55:04,56510551,84041264.26,0.002856,COP,239999.999998,19426.720703,55.477663
42636058,2022-09-28 01:55:10,56510558,18029.58,0.031892,USD,575.000087,19426.720703,619.557754
42636059,2022-09-28 01:55:10,56510556,21699.47,0.005530,USD,120.000022,19426.720703,107.431514
42636060,2022-09-28 01:57:02,56510553,20045.22,0.002494,USD,49.999995,19426.720703,48.457235


## Prepare data for algorithm

In [8]:
# trades.set_index('date', inplace=True)
trades['counter'] = 1
trades

,date,tid,price,amount,Currency,amount_nominal,Close,amount_usd,counter
0,2017-03-15 00:00:13,6277278,26490.07,0.075500,MXN,2000.000285,1249.609985,94.345554,1
1,2017-03-15 00:00:15,100281,4126984.13,0.031500,VEF,130000.000095,1249.609985,39.362715,1
2,2017-03-15 00:00:15,6277265,1292.97,0.657400,USD,849.998478,1249.609985,821.493604,1
3,2017-03-15 00:00:24,6277292,1068.00,0.110000,GBP,117.480000,1249.609985,137.457098,1
4,2017-03-15 00:00:40,6277296,1062.93,0.117600,GBP,125.000568,1249.609985,146.954134,1
...,...,...,...,...,...,...,...,...,...
42636057,2022-09-28 01:55:04,56510551,84041264.26,0.002856,COP,239999.999998,19426.720703,55.477663,1
42636058,2022-09-28 01:55:10,56510558,18029.58,0.031892,USD,575.000087,19426.720703,619.557754,1
42636059,2022-09-28 01:55:10,56510556,21699.47,0.005530,USD,120.000022,19426.720703,107.431514,1
42636060,2022-09-28 01:57:02,56510553,20045.22,0.002494,USD,49.999995,19426.720703,48.457235,1


In [13]:
path = '../temporary/trades_cleaned.csv'
trades.to_csv(path, index=False)

# Paxful

In [3]:
trades_paxful = pd.read_csv('../temporary/trades_paxful_combined.csv')
trades_paxful

,id,date,amount,price,payment_method,payment_method_group,currency,type,advertiser_cc,user_cc,crypto_rate_usd,crypto_code,duration
0,4544611,1489557966,0.009774,20.0,OneVanilla VISA/MasterCard Gift Card,Gift cards,CAD,buy,CA,CA,1246.86,BTC,367
1,4544613,1489557927,0.094035,150.0,iTunes Gift Card,Gift cards,USD,buy,CN,ZA,1246.86,BTC,317
2,4544617,1489557928,0.595860,1300.0,Amazon Gift Card,Gift cards,USD,sell,US,UZ,1246.86,BTC,297
3,4544626,1489558541,0.217414,450.0,Walmart Gift Card,Gift cards,USD,buy,VN,NG,1246.86,BTC,816
4,4544627,1489557866,0.057287,100.0,PayPal,Online wallets,USD,buy,CM,US,1246.86,BTC,141
...,...,...,...,...,...,...,...,...,...,...,...,...,...
89670080,200223982,1664431440,0.000479,10.0,Dogecoin DOGE,Digital currencies,USD,buy,VN,VN,19425.39,BTC,410
89670081,200223983,1664431429,0.001549,50.0,OneVanilla VISA/MasterCard Gift Card,Gift cards,CAD,buy,NaN,US,19425.39,BTC,397
89670082,200223984,1664431434,0.000479,10.0,Dogecoin DOGE,Digital currencies,USD,buy,VN,VN,19425.39,BTC,402
89670083,200223985,1664431313,0.000531,20.0,Steam Wallet Gift Card,Gift cards,AUD,buy,US,NG,19425.39,BTC,280


In [4]:
trades_paxful = trades_paxful.drop(columns = ['duration', 'crypto_code'])
trades_paxful = trades_paxful.rename(columns = {'id': 'tid'})
trades_paxful['amount_usd'] = trades_paxful['amount']*trades_paxful['crypto_rate_usd']
trades_paxful['date'] = pd.to_datetime(trades_paxful['date'], unit = 's')
trades_paxful

,tid,date,amount,price,payment_method,payment_method_group,currency,type,advertiser_cc,user_cc,crypto_rate_usd,amount_usd
0,4544611,2017-03-15 06:06:06,0.009774,20.0,OneVanilla VISA/MasterCard Gift Card,Gift cards,CAD,buy,CA,CA,1246.86,12.186423
1,4544613,2017-03-15 06:05:27,0.094035,150.0,iTunes Gift Card,Gift cards,USD,buy,CN,ZA,1246.86,117.248530
2,4544617,2017-03-15 06:05:28,0.595860,1300.0,Amazon Gift Card,Gift cards,USD,sell,US,UZ,1246.86,742.954187
3,4544626,2017-03-15 06:15:41,0.217414,450.0,Walmart Gift Card,Gift cards,USD,buy,VN,NG,1246.86,271.085331
4,4544627,2017-03-15 06:04:26,0.057287,100.0,PayPal,Online wallets,USD,buy,CM,US,1246.86,71.428732
...,...,...,...,...,...,...,...,...,...,...,...,...
89670080,200223982,2022-09-29 06:04:00,0.000479,10.0,Dogecoin DOGE,Digital currencies,USD,buy,VN,VN,19425.39,9.299323
89670081,200223983,2022-09-29 06:03:49,0.001549,50.0,OneVanilla VISA/MasterCard Gift Card,Gift cards,CAD,buy,NaN,US,19425.39,30.082936
89670082,200223984,2022-09-29 06:03:54,0.000479,10.0,Dogecoin DOGE,Digital currencies,USD,buy,VN,VN,19425.39,9.299323
89670083,200223985,2022-09-29 06:01:53,0.000531,20.0,Steam Wallet Gift Card,Gift cards,AUD,buy,US,NG,19425.39,10.324206


In [5]:
trades_paxful = trades_paxful.sort_values(by='date')
trades_paxful = trades_paxful.drop_duplicates(subset = ['tid'])
trades_paxful

,tid,date,amount,price,payment_method,payment_method_group,currency,type,advertiser_cc,user_cc,crypto_rate_usd,amount_usd
6,4544629,2017-03-15 06:04:04,0.013070,20.00,"Any Visa, MasterCard and AmEx Gift Cards",Gift cards,USD,buy,US,US,1246.86,16.295949
4,4544627,2017-03-15 06:04:26,0.057287,100.00,PayPal,Online wallets,USD,buy,CM,US,1246.86,71.428732
11,4544639,2017-03-15 06:04:39,0.016885,25.00,Amazon Gift Card,Gift cards,USD,buy,US,US,1247.49,21.063357
1,4544613,2017-03-15 06:05:27,0.094035,150.00,iTunes Gift Card,Gift cards,USD,buy,CN,ZA,1246.86,117.248530
2,4544617,2017-03-15 06:05:28,0.595860,1300.00,Amazon Gift Card,Gift cards,USD,sell,US,UZ,1246.86,742.954187
...,...,...,...,...,...,...,...,...,...,...,...,...
89238263,199258589,2022-10-23 11:04:25,0.001452,19600.00,International Wire Transfer (SWIFT),Bank transfers,NGN,sell,NG,NG,18967.26,27.545393
89285150,199372404,2022-10-28 17:59:12,0.004265,1450.03,Bank Transfer,Bank transfers,ZAR,sell,ZA,ZA,19394.79,82.711021
88428992,197433205,2022-10-28 20:13:13,0.000697,9600.00,Bank Transfer,Bank transfers,NGN,sell,NG,NG,19732.57,13.762284
88316544,197162038,2022-10-28 20:14:42,0.000728,10000.00,Bank Transfer,Bank transfers,NGN,sell,NG,NG,19801.36,14.425291


In [7]:
#remove small number of cc's with ".1"
trades_paxful['user_cc'] = trades_paxful['user_cc'].str.replace('.1','')
trades_paxful['user_cc'] = trades_paxful['user_cc'].str.replace('SK', 'CZ')

C:\Users\Austin\AppData\Local\Temp\ipykernel_7072\2885858085.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  trades_paxful['user_cc'] = trades_paxful['user_cc'].str.replace('.1','')


In [8]:
#Get country codes
paxful_cc = trades_paxful["user_cc"].unique()
paxful_cc

array(['US', 'ZA', 'UZ', 'CA', 'NG', 'BR', 'NL', 'SV', 'GH', 'NO', 'DE',
       'EG', 'FR', 'GB', 'MZ', 'IN', 'XX', 'AE', 'CM', 'AU', 'VE', 'RO',
       'KE', 'BF', 'IL', 'UA', 'CZ', 'PH', 'RU', 'IT', 'BD', 'IQ', 'TR',
       'BG', 'GR', 'TH', 'EE', 'IM', 'PS', 'JO', 'VN', 'BO', 'LY', 'TN',
       'BJ', 'ID', 'MY', 'CN', 'ES', 'DO', 'AR', 'FI', 'TZ', 'AL', 'SI',
       'PK', 'NZ', 'RS', 'MM', 'LT', 'KH', 'PR', 'HU', 'CH', 'KR', 'SE',
       'IE', 'PE', 'AT', 'LB', 'MX', 'HR', 'BE', 'EC', 'CL', 'LV', 'TT',
       'KW', 'CO', 'MD', 'LU', 'PL', 'DZ', 'PT', 'SG', 'DK', 'CD', 'MA',
       'SA', 'BY', 'BA', 'JP', 'ZW', 'ZM', 'GE', 'CR', 'MK', 'HK', 'SC',
       'BH', 'JM', 'BB', 'SN', 'YT', 'CI', 'LK', 'RE', 'YE', 'MT', 'ML',
       'AM', 'GM', 'GY', 'IR', 'TW', 'UY', 'HT', 'PA', 'CC', 'QA', 'BS',
       'UG', 'NI', 'TG', 'MR', 'NP', 'MV', 'CY', nan, 'AF', 'BW', 'PY',
       'PF', 'LR', 'SL', 'GT', 'VC', 'PG', 'BM', 'VG', 'KZ', 'PW', 'MC',
       'AG', 'SZ', 'IS', 'OM', 'KN', 'MU', 'AS', 'GU

In [9]:
#export stuff

path = '../temporary/trades_paxful_cleaned.csv'
trades_paxful.to_csv(path, index = False)

In [10]:
paxful_cc = pd.DataFrame(paxful_cc, columns = ["cc"])

paxful_cc.to_csv("../temporary/paxful_cc.csv", index = False)

## Playground

In [23]:
trades_paxful[trades_paxful["user_cc"] == "US.1"]

,tid,date,amount,price,payment_method,payment_method_group,currency,type,advertiser_cc,user_cc,crypto_rate_usd,amount_usd
341106,5202253,2017-04-18 13:13:04,0.014816,20.00,Prepaid Debit Card,Debit/credit cards,USD,buy,US,US.1,1226.20,18.167539
1989938,7087059,2017-07-03 22:36:40,0.005819,20.00,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,US,US.1,2563.48,14.916480
3373565,11332570,2017-11-04 18:44:40,0.001362,25.00,Gift Cards (Miscellaneous Retailers),Gift cards,USD,buy,US,US.1,7337.83,9.992950
3463266,11436363,2017-11-07 03:39:14,0.010812,99.99,PayPal,Online wallets,USD,buy,US,US.1,7114.20,76.915387
5012296,14217644,2018-01-19 20:06:08,0.002490,50.00,GameStop Gift Card,Gift cards,USD,buy,US,US.1,11203.69,27.901782
5421603,15401757,2018-02-20 19:53:55,0.012564,300.00,iTunes Gift Card,Gift cards,CAD,buy,US,US.1,11612.93,145.903807
5439403,15532449,2018-02-24 00:08:10,0.002088,30.00,Amazon Gift Card,Gift cards,USD,buy,US,US.1,10114.59,21.117241
5640706,15922615,2018-03-04 17:35:37,0.008453,100.00,Bluebird American Express,Debit/credit cards,USD,buy,US,US.1,11117.10,93.977738
5887208,16398090,2018-03-15 18:07:28,0.003887,39.00,OneVanilla VISA/MasterCard Gift Card,Gift cards,USD,buy,US,US.1,8150.07,31.683316
6461615,17487683,2018-04-07 21:16:47,0.004339,40.00,Amazon Gift Card,Gift cards,USD,buy,US,US.1,6983.03,30.297412


In [9]:
trades_paxful.dtypes

tid                              int64
date                    datetime64[ns]
amount                         float64
price                          float64
payment_method                  object
payment_method_group            object
currency                        object
type                            object
advertiser_cc                   object
user_cc                         object
crypto_rate_usd                float64
amount_usd                     float64
dtype: object

In [9]:
margins = (trades_paxful['price'] - (trades_paxful['crypto_rate_usd']*trades_paxful['amount']))/trades_paxful['price']

In [10]:
margins.describe()

count    8.967008e+07
mean     4.991155e-01
std      3.200819e-01
min     -6.172700e+02
25%      2.826252e-01
50%      3.742091e-01
75%      8.456629e-01
max      1.000000e+00
dtype: float64